In [65]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,  chi2
from sklearn import svm
from scipy.sparse import csr_matrix


In [66]:
dom1 = []
dom2 = []
test_set = []
with open('data/domain1_train.json', 'r') as file:
    for line in file:
        dom1.append(json.loads(line))
        
with open('data/domain2_train.json', 'r') as file:
    for line in file:
        dom2.append(json.loads(line))
        
sam = pd.read_csv("data/sample.csv")

with open('data/test_set.json', 'r') as file:
    for line in file:
        test_set.append(json.loads(line))
        
dom1 = pd.DataFrame.from_dict(dom1)
dom2 = pd.DataFrame.from_dict(dom2)
dom2 = dom2[dom2['text'].apply(len) > 0]
test_set = pd.DataFrame.from_dict(test_set)
        

In [67]:
print(len(dom1))
print(len(dom2))
print(len(sam))
print(len(test_set))

19500
14899
1000
1000


In [68]:
dom1['label'].value_counts()
print(len(dom2.loc[dom2['label']==0]))
print(len(dom2.loc[dom2['label']==1]))

12750
2149


In [69]:

dom2_0 = dom2.loc[dom2['label']==0]
dom2_1 = dom2.loc[dom2['label']==1]

length_dom2_1 = len(dom2_1)
mol_num = len(dom2['model'].value_counts())
sam_dom2_0 = dom2_0.groupby('model').apply(lambda x: x.sample(n=int(length_dom2_1/mol_num)))
sam_dom2_0.reset_index(drop=True, inplace=True)
dom2_new = result = pd.concat([dom2_1, sam_dom2_0])

sampled_dom1 = dom1.groupby('label').apply(lambda x: x.sample(n=int(length_dom2_1/mol_num)))
sampled_dom1.reset_index(drop=True, inplace=True)

data = result = pd.concat([sampled_dom1, dom2_new])

In [70]:
from sklearn.model_selection import train_test_split
data['model'].fillna(8, inplace=True)


X = data[['text','model']] 
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=data[['model', 'label']], random_state=42)

In [71]:
y_train

1640    0
227     0
631     0
1887    0
1576    0
       ..
1445    0
2014    0
694     1
360     1
374     0
Name: label, Length: 3929, dtype: int64

In [72]:
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

print(X_train['model'].value_counts(normalize=True))
print(X_test['model'].value_counts(normalize=True))

label
0    0.500127
1    0.499873
Name: proportion, dtype: float64
label
1    0.500509
0    0.499491
Name: proportion, dtype: float64
model
8.0    0.562484
5.0    0.062611
0.0    0.062611
2.0    0.062611
1.0    0.062611
6.0    0.062357
4.0    0.062357
3.0    0.062357
Name: proportion, dtype: float64
model
8.0    0.562564
4.0    0.063072
6.0    0.063072
3.0    0.063072
2.0    0.062055
0.0    0.062055
5.0    0.062055
1.0    0.062055
Name: proportion, dtype: float64


In [73]:
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np


X = X_train['text'].tolist()
X1 =X_test['text'].tolist()
max_length = max([len(seq) for seq in X])
word2vec = Word2Vec(X, vector_size=300, window=10, min_count=1, workers=4)

def get_text_vector(text, word2vec_model):
    vector_list = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv.index_to_key]
    if len(vector_list) == 0:
        return np.zeros(word2vec.vector_size)
    return np.mean(vector_list, axis=0)

X_train = np.array([get_text_vector(text, word2vec) for text in X])
X_test = np.array([get_text_vector(text, word2vec) for text in X1])

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np





In [74]:
print(y_train.value_counts(normalize=True))

label
0    0.500127
1    0.499873
Name: proportion, dtype: float64


In [75]:
model = svm.SVC()
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Test score: {score}")

Test score: 0.6948118006103764


In [76]:
X2 =test_set['text'].tolist()
test_set = np.array([get_text_vector(text, word2vec) for text in X2])
test_pre = model.predict(test_set)
result = pd.DataFrame({
    'class': test_pre,
})
result['id'] = result.index
result = result[['id'] + [col for col in result.columns if col != 'id']]
result

,id,class
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
995,995,0
996,996,1
997,997,0
998,998,1


In [77]:
result.to_csv('result1.csv', index=False)
